# *Cosmian Secure Computation:* Run any Python code on encrypted data

## Overview

On April 15, 1912, the Titanic sank after colliding with an iceberg. Unfortunately, there weren’t enough lifeboats for everyone on board, resulting in the death of 1502 out of 2224 passengers and crew.

Alice and Bob each have half of the passenger data (name, age, gender, socio-economic class, etc.) and want to collaborate on statistical analysis without sharing their data.

*Cosmian Secure Computation* lets them do this analysis while their data stay encrypted the whole time.


## Step 1: Create computation (Computation Owner)

The first step is to create a computation and specify each role.

In this example, Alice, the `Computation Owner`, is also the `Code Provider`, one of the `Data Providers`, and the `Result Consumer`.

Bob, aka `Data Provider 2` and `Result Consumer 2`, will provide his data.

In [ ]:
%pip install cosmian_secure_computation_client>=0.4.2,<0.5.0 --quiet --upgrade
%pip install seaborn --quiet
%pip install pandas --quiet
import seaborn as sns
import pandas as pd
import time

from cosmian_secure_computation_client import ComputationOwnerAPI, CodeProviderAPI, DataProviderAPI, ResultConsumerAPI
from cosmian_secure_computation_client import CryptoContext
from cosmian_secure_computation_client.log import setup_logging
from importlib.metadata import version
from IPython.core.display import HTML
from configparser import ConfigParser
from pathlib import Path 

print("Cosmian Secure Computation Client version " + version('cosmian_secure_computation_client'))

# Secret tokens are loaded from the config.ini file
config = ConfigParser()
config.read("config.ini")
secret_token_alice = config["Credentials"]["token_1"]
secret_token_bob = config["Credentials"]["token_2"]

# Setup logging on the notebook
setup_logging(debug=True)
HTML("""<style>.jp-RenderedText[data-mime-type='application/vnd.jupyter.stderr'] {background:#d3f8d3 !important;}</style>""")

In [ ]:
# The Computation Owner creates a new computation
computation_owner = ComputationOwnerAPI(secret_token_alice)
computation = computation_owner.create_computation(
    'Titanic 🌊 v0.4',
    code_provider_email="alice@cosmian.com",
    data_providers_emails=["alice@cosmian.com", "bob@cosmian.com"],
    result_consumers_emails=["alice@cosmian.com", "bob@cosmian.com"])

---
## Step 2: Register (All participants)

To register, participants need two things:
1) **A secret token**. To get a secret token, participants must create an account on the [Cosmian console](https://console.cosmian.com) and go to the [Secret token page](https://console.cosmian.com/secret-token). In this example, the tokens are stored in the `config.ini` file. They were loaded in the previous cell.

2) **A CryptoContext**. This object is generated client-side and contains the keys to encrypt and decrypt the code, the data, and the result.

The CryptoContext also contains a **pre-shared secret**. It is a list of 3 words randomly picked from the BIP39 list. The Computation Owner generates this secret on their side and transfers it to the other participants only. The computation's participants only know them, and Cosmian will never ask for them.

In [ ]:
# The Computation Owner generates a list of three random words
words = ComputationOwnerAPI.random_words()

# The computation UUID is stored inside a variable
computation_uuid = computation.uuid

from cosmian_secure_computation_client import Side

# The Code Provider registers
code_provider_ctx = CryptoContext(computation_uuid=computation_uuid, side=Side.CodeProvider, words=words)
code_provider = CodeProviderAPI(token=secret_token_alice, ctx=code_provider_ctx)
code_provider.register(computation_uuid)


# The Data Providers register
data_provider1_ctx = CryptoContext(computation_uuid=computation_uuid, side=Side.DataProvider, words=words)
data_provider1 = DataProviderAPI(token=secret_token_alice, ctx=data_provider1_ctx)
data_provider1.register(computation_uuid)

data_provider2_ctx = CryptoContext(computation_uuid=computation_uuid, side=Side.DataProvider, words=words)
data_provider2 = DataProviderAPI(token=secret_token_bob, ctx=data_provider2_ctx)
data_provider2.register(computation_uuid)


# The Result Consumers register
result_consumer1_ctx = CryptoContext(computation_uuid=computation_uuid, side=Side.ResultConsumer, words=words)
result_consumer1 = ResultConsumerAPI(token=secret_token_alice, ctx=result_consumer1_ctx)
result_consumer1.register(computation_uuid)

result_consumer2_ctx = CryptoContext(computation_uuid=computation_uuid, side=Side.ResultConsumer, words=words)
result_consumer2 = ResultConsumerAPI(token=secret_token_bob, ctx=result_consumer2_ctx)
result_consumer2.register(computation_uuid)

---
## Step 3: Provide code (Code Provider)

Alice provides the code she wrote. It must be organized like this:

```bash
code_titanic
├── secret_module.py
└── run.py
```

The entire folder (except `run.py`) is encrypted with the CryptoContext's symmetric key before being sent to the enclave.

Once the secure enclave receives the code, it is sealed and generates the computation checklist automatically.

In [ ]:
# The Code Provider specifies the path of their Python code directory
path = Path("./code_titanic")

# The Code Provider sends their code. It is encrypted on the client-side
code_provider.upload_code(computation_uuid, path);

Once the enclave is generated, Alice can send her symmetric key, sealed with the public key of the enclave.

In [ ]:
# The Code Provider waits for the generation of the enclave identity
enclave_public_key = code_provider.wait_for_enclave_identity(computation_uuid)

In [ ]:
# The Code Provider sends their sealed symmetric key
computation = code_provider.get_computation(computation_uuid)
code_provider.key_provisioning(computation_uuid, computation.enclave.identity.public_key)

---
## Step 4: Send data (Data Providers)

Alice and Bob, the `Data Providers`, can now provide their data. As for the code, data are encrypted with symmetric keys before sending them.

Alice provides her dataset (`titanic_1.csv`) and Bob his (`titanic_2.csv`). Once the data are sent, Alice and Bob can send their symmetric keys, sealed with the public key of the enclave.

In [ ]:
# Load and split the Titanic dataset from the seaborn library
titanic = sns.load_dataset('titanic')
titanic_1=titanic.sample(frac=0.5)
titanic_2=titanic.drop(titanic_1.index)
titanic_1.to_csv("data/titanic_1.csv")
titanic_2.to_csv("data/titanic_2.csv")


# The Data Provider 1 sends their data
path_1 = Path("data/titanic_1.csv")
data_provider1.upload_files(computation_uuid, [path_1])
data_provider1.done(computation_uuid)


# The Data Provider 2 sends their data
path_2 = Path("data/titanic_2.csv")
data_provider2.upload_files(computation_uuid, [path_2])
data_provider2.done(computation_uuid)

Once the data are sent, Alice and Bob can send their symmetric keys, also sealed with the public key of the enclave.

In [ ]:
# The Data Provider 1 sends their sealed symmetric key
data_provider1.key_provisioning(computation_uuid, computation.enclave.identity.public_key)

# The Data Provider 2 sends their sealed symmetric key
data_provider2.key_provisioning(computation_uuid, computation.enclave.identity.public_key)

---
## Step 5: Run computation (Result Consumers)

Finally, Alice and Bob, who also are `Result Consumers`, will send their sealed symmetric keys. They will use that key later to decrypt the result.

Again, these symmetric keys are sealed with the public key of the enclave before being sent.

Once the enclave receives the sealed keys, the computation starts automatically.

In [ ]:
# The Result Consumer 1 sends their sealed symmetric key
result_consumer1.key_provisioning(computation_uuid, computation.enclave.identity.public_key)

# The Result Consumer 2 sends their sealed symmetric key
result_consumer2.key_provisioning(computation_uuid, computation.enclave.identity.public_key)

---
## Step 6: Get result (Result Consumers)

Once the run ends, the computation returns the exit code (`0` if everything worked, non-zero if there was an error) and the standard console outputs (`stdout` and `stderr`).

If the exit code is `0`, Alice and Bob can fetch their results and decrypt them with their symmetric keys.

In [ ]:
# The Result Consumer 1 waits for the result and fetches it. The result is decrypted automatically with the key stored in the CryptoContext
result = result_consumer1.wait_result(computation_uuid)

In [ ]:
# The result is written on the disc
result_path: Path = Path("data/titanic_result.csv")
result_path.write_bytes(result)

# The result is deserialized and plotted on the Notebook
df = pd.read_csv(result_path)
df.rename(columns={'1':'Survival rate'}, inplace=True)
sns.catplot(x="class", y="Survival rate", col="who",data=df, kind="bar", palette="pastel")